# The Three Body Problem: a numerical visualization in 2D

A analytic mechanics problem with no closed solution: three bodies of comparable masses orbiting each other, a chaotic system with a few stable solutions. In this notebook a visualization is made in 2D, however, the same formulas would apply to a 3D scenario. 

The scene is governed by Newton's laws of motion, each body has a mass $(m)$, and initial conditions for position $(\vec{r}_0)$, and velocity $(\vec{v}_0)$. From these we can solve the acceleration based on Newton's gravitational law and integrate to calculate the position of each body after a time $dt$.

## Newton's Law of Gravity
$$
\mathbf{F}_{12}
=
-\,G\,\frac{m_1 m_2}{\lVert \mathbf{r}_1 - \mathbf{r}_2 \rVert^3}
\left( \mathbf{r}_1 - \mathbf{r}_2 \right)
$$


## Acceleration for each body

\begin{align}
\ddot{\mathbf{r}}_1 &=
- G \left( m_2 \frac{\mathbf{r}_1 - \mathbf{r}_2}{\lVert \mathbf{r}_1 - \mathbf{r}_2 \rVert^3}
+ m_3 \frac{\mathbf{r}_1 - \mathbf{r}_3}{\lVert \mathbf{r}_1 - \mathbf{r}_3 \rVert^3} \right), \\[1em]

\ddot{\mathbf{r}}_2 &=
- G \left( m_3 \frac{\mathbf{r}_2 - \mathbf{r}_3}{\lVert \mathbf{r}_2 - \mathbf{r}_3 \rVert^3}
+ m_1 \frac{\mathbf{r}_2 - \mathbf{r}_1}{\lVert \mathbf{r}_2 - \mathbf{r}_1 \rVert^3} \right), \\[1em]

\ddot{\mathbf{r}}_3 &=
- G \left( m_1 \frac{\mathbf{r}_3 - \mathbf{r}_1}{\lVert \mathbf{r}_3 - \mathbf{r}_1 \rVert^3}
+ m_2 \frac{\mathbf{r}_3 - \mathbf{r}_2}{\lVert \mathbf{r}_3 - \mathbf{r}_2 \rVert^3} \right).
\end{align}



In [9]:
%%manim -ql -v WARNING ThreeBodyProblem
from manim import *
import pandas as pd
import os
import numpy as np
os.environ["PATH"] = "/Library/TeX/texbin:" + os.environ["PATH"]


G = 1.0  # Gravitational constant (unit for simplicity)

class ThreeBodyProblem(Scene):
    def construct(self):

        choices = [
            "all_diverge",
            "figure_8",
            "weird_spiral",
            "chaotic_but_bounded"
        ]
        choice = choices[3]

        if choice == "all_diverge": 
        # ----------- ALL DIVERGE ------------------
            M = np.array([1.0, 1.0, 1.0])
            dt = 0.01
            self.R = np.array([
                [-1.0,  0.0],
                [ 1.0,  0.0],
                [ 0.0,  1.5]
            ])
            self.V = np.array([
                [ 0.3,  0.5],
                [-0.3,  0.5],
                [ 0.0, -0.6]
            ])
        elif choice == "figure_8":

        # ----------- FIGURE-8 ------------------
            M = np.array([1.0, 1.0, 1.0])
            dt = 0.01

            self.R = np.array([
                [-0.97000436,  0.24308753],
                [ 0.97000436, -0.24308753],
                [ 0.0,         0.0       ]
            ])

            self.V = np.array([
                [ 0.4662036850,  0.4323657300],
                [ 0.4662036850,  0.4323657300],
                [-0.93240737,   -0.86473146 ]
            ])
        elif choice == "weird_spiral":
        # ----------- WEIRD SPIRAL ------------------
            M = np.array([1.0, 1.0, 1.0])
            dt = 0.01

            L = 1.0
            v = 0.6

            self.R = np.array([
                [-L,  0.0],
                [ L,  0.0],
                [ 0.0,  np.sqrt(3)*L]
            ])

            self.V = np.array([
                [  v,  0.0],
                [  v,  0.0],
                [ -2*v, 0.0]
            ])
        else:
        # ----------- CHAOTIC BUT BOUNDED (ROBUST) ------------------
            M = np.array([1.0, 1.0, 1.0])
            dt = 0.005   # important: smaller timestep

            self.R = np.array([
                [-1.0,  0.0],
                [ 1.0,  0.0],
                [ 0.0,  0.8]
            ])

            self.V = np.array([
                [ 0.0,  0.25],
                [ 0.0, -0.30],
                [ 0.0,  0.00]
            ])

        
        colors = [RED, BLUE, GREEN]
        bodies = VGroup()

        for i in range(3):
            dot = Dot(
                point=[self.R[i, 0], self.R[i, 1], 0],
                color=colors[i]
            )
            bodies.add(dot)

        self.add(bodies)

        trails = [
            TracedPath(
                bodies[i].get_center,
                stroke_color=colors[i],
                stroke_width=2
            )
            for i in range(3)
        ]
        self.add(*trails)

        # -----------------------------
        
        def acceleration(pos):
            acc = np.zeros_like(pos)
            for i in range(3):
                for j in range(3):
                    if i != j:
                        r = pos[j] - pos[i]
                        dist = np.linalg.norm(r) + 1e-5
                        acc[i] += G * M[j] * r / dist**3
            return acc

        # -----------------------------
        # Integrals using the acc calculated function
        def step_movement(pos, vel, dt):
            a1 = acceleration(pos)
            k1v = a1 * dt
            k1x = vel * dt

            a2 = acceleration(pos + 0.5 * k1x)
            k2v = a2 * dt
            k2x = (vel + 0.5 * k1v) * dt

            a3 = acceleration(pos + 0.5 * k2x)
            k3v = a3 * dt
            k3x = (vel + 0.5 * k2v) * dt

            a4 = acceleration(pos + k3x)
            k4v = a4 * dt
            k4x = (vel + k3v) * dt

            pos_new = pos + (k1x + 2*k2x + 2*k3x + k4x) / 6
            vel_new = vel + (k1v + 2*k2v + 2*k3v + k4v) / 6

            return pos_new, vel_new

        # -----------------------------
        # Manim updater
        def update_system(mob, dt_manim=0):
            self.R, self.V = step_movement(self.R, self.V, dt)
            for i, body in enumerate(bodies):
                body.move_to([self.R[i, 0], self.R[i, 1], 0])

        bodies.add_updater(update_system)

        self.wait(30)

Manim Community v0.19.1